In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
%matplotlib inline

In [ ]:
def tryfun(f):
    def wrapper(i):
        try:
            return f(i)
        except:
            return np.NaN
    return wrapper

# Outputs


In [ ]:
notifs = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Notificaciones TB/GTM - TB all notifications 2012-Sep2018.csv")

In [ ]:
notifs.loc[notifs.COD_MUNI == 1422, "COD_MUNI"] = 1420

In [ ]:
notifs["Year"] = np.floor(notifs.YearMonth/100)

In [ ]:
casesNotif = notifs[(notifs.Year<2018) & notifs.COD_MUNI.notna()].assign(municipality = lambda x: x.COD_MUNI).assign(date=lambda x: x.Year.astype(int))\
.assign(department = lambda x: (np.floor(x.COD_MUNI/100)).astype(int) ).groupby(["date", "department", "municipality"]).apply(len)

In [ ]:
casesNotif = casesNotif[casesNotif.index.get_level_values(1)!=0].rename("Cases Notified_value_m").fillna(0)

In [ ]:
extraM2017 = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Extramuros2017.csv", index_col=0)

In [ ]:
acfcases = extraM2017[extraM2017.Year.notna()].assign(municipality = lambda x: x.MuniCode.astype(int)).assign(date = lambda x: x.Year.astype(int)).\
assign(department = lambda x: x.DeptoCode.astype(int)).groupby(["date", "department", "municipality"])["bacteriologicos.extramuros"].sum()
acfcases = acfcases.rename("Additional Cases Detected via ACF_value_m").fillna(0)

In [ ]:
#acfcases = acfcases.reindex(casesNotif.index)

In [ ]:
# Cases Started on Treatment_value
# MDR Cases Started Treatment_value

cohortes = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/COHORTES/GTM - Tx cohort data 2012-2016.csv")

In [ ]:
cohortes.head()

In [ ]:
pivot_cohortes = cohortes[(cohortes.col_name.isin(["TOTAL"])) & 
       (cohortes.table.isin(["Nuevos Pulmonares BK+", "Nuevos Pulmonares BK-", "Nuevos Pediatricos", 
                          "Nuevos Extrapulmonares", "Nuevos TB/VIH", "Retratamiento"])) &
       (cohortes.deptocode!=0)].\
    groupby(["year", "deptocode", "row_name_B"]).value.sum().unstack(2)

In [ ]:
pivot_cohortes.head()

In [ ]:
initiatedTx = pivot_cohortes.sum(axis=1)

In [ ]:
initiatedTx = initiatedTx.rename_axis(index = { "year": "date", "deptocode": "department" }).rename("Cases Started on Treatment_value_d")

In [ ]:
initiatedTx2 = initiatedTx.reindex(casesNotif.index).fillna(0)

In [ ]:
# Reindex is not bad in this case??
initiatedTx.sum(), initiatedTx2.groupby([initiatedTx2.index.get_level_values(0), initiatedTx2.index.get_level_values(1)]).first().sum()

In [ ]:
tbquimio = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Notificaciones TB/GTM - TB quimio 2012-Sep2018.csv",
                     index_col = 0, low_memory = False)

In [ ]:
tbquimio.head()

In [ ]:
tbquimio["Year"] = np.floor(tbquimio.YearMonth/100)

In [ ]:
plhivipt= tbquimio[tbquimio.VIH=="Reactive"].assign(department = lambda x: np.floor(x.COD_MUNI/100))\
.groupby(["Year", "department", "COD_MUNI"]).apply(len)

In [ ]:
plhivipt = plhivipt.rename("PLHIV started on IPT_value_m").fillna(0)\
.rename_axis(index={"Year": "date", "COD_MUNI": "municipality"}) #.reindex(casesNotif.index)

In [ ]:
# This is only contacts in notifications database. I am not sure if its ok.

In [ ]:
childipt= tbquimio[(tbquimio.Year<2018) & (tbquimio.EDAD <= 5)].assign(department = lambda x: np.floor(x.COD_MUNI/100))\
.groupby(["Year", "department", "COD_MUNI"]).apply(len).rename("Children in Contact with TB Started IPT_value_m")

In [ ]:
childipt = childipt.rename_axis(index={"Year": "date", "COD_MUNI": "municipality"}) #.reindex(casesNotif.index)

In [ ]:
# Cases Notified in Prisons_value
pplcases = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/PPL-conteos-2014-2017.csv", index_col=0)

In [ ]:
pplcases = pplcases.assign(department = lambda x : np.floor(x.municipio/100)).rename(columns=\
{"n": "Cases Notified in Prisons_value", "YEAR": "date", "municipio": "municipality"}).set_index(["date", "department", "municipality"])

In [ ]:
pplcases = pplcases.reindex(casesNotif.index)

In [ ]:
ppldata = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Prisoners/tb-ppl-2014-2017.csv")

In [ ]:
ppldata["TXDATE"] = pd.to_datetime(ppldata.TXDATE, errors="coerce")

In [ ]:
# Sólo una persona no recibió tratamiento.
ppldata.TXDATE.isna().sum()

In [ ]:
ppltx = ppldata[ppldata.TXDATE.notna()].assign(department = lambda x: np.floor(x.municode/100).astype(int) )\
.assign(date = lambda x: x.TXDATE.map(lambda y: y.year ))\
.groupby(["date", "department" , "municode"]).apply(len)\
.rename_axis(index={"municode": "municipality"})\
.rename("Cases Started on Treatment in Prisons_value_m")

In [ ]:
ppltx = ppltx[ppltx.index.get_level_values("date")>2012]
ppltx

In [ ]:
# Lets see in the cohort data:
hivyear_A = cohortes[(cohortes.col_name.isin(["TOTAL"])) & 
       (cohortes.table.isin(["Nuevos TB/VIH"])) &
       (cohortes.deptocode==0)].groupby("year").value.sum()
hivyear_A

In [ ]:
# Now with notifications data:
hivyear_B = notifs.groupby(["YEAR", "VIH"]).apply(len).unstack(1)["Reactive"]
hivyear_B

In [ ]:
# HIV counts per year, from cohort and from notifications seems to be worse than in previous years
# This may be interpreted as the percentage of HIV cases that received treatment.
hivyear_A/hivyear_B

In [ ]:
# The actual data to integrate into the tables:
# HIV/TB Cases Notified_value

hivcases = notifs[(notifs.VIH == "Reactive") & (notifs.Year<2018) & notifs.COD_MUNI.notna()]\
.assign(municipality = lambda x: x.COD_MUNI.astype(int)).assign(date=lambda x: x.Year.astype(int))\
.assign(department = lambda x: (np.floor(x.COD_MUNI/100)).astype(int) ).groupby(["date", "department", "municipality"]).apply(len)\
.rename("HIV/TB Cases Notified_value_m")

In [ ]:
# Now looking at MDR data in notifications:
notifs[notifs.DBCATEGORY=="MDR"].YEAR.value_counts()

In [ ]:
# There is actually a MDR table with all available cohort data:
mdrtb = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/COHORTES/GTM - TB MDR Cohort Data 2014-2016.csv")

In [ ]:
mdrtb.head()

In [ ]:
mdrtb.Year.value_counts()

In [ ]:
# How many bad tx initiation date values are there?
mdrtb.groupby("Year").apply(lambda x: x.TxDate_Year.isna().sum())

In [ ]:
# MDR Cases Notified_value
# MDR Cases Started Treatment_value

mdrcases = mdrtb[mdrtb.deptocode.notna()].assign(date = lambda x: x.Year+2000).assign(department = lambda x: x.deptocode.astype(int))\
    .groupby(["date","department"]).apply(len).rename("MDR Cases Notified_value_d")
    
mdrtx = mdrtb[mdrtb.deptocode.notna()].assign(date = lambda x: x.TxDate_Year.map(tryfun(int)) ).assign(department = lambda x: x.deptocode.astype(int))\
    .groupby(["date","department"]).apply(len).rename("MDR Cases Started Treatment_value_d")

mdrtx = mdrtx[mdrtx.index.get_level_values(0)>2012]

In [ ]:
outputs = pd.merge(casesNotif, acfcases, left_index=True, right_index=True, how="outer")\
            .merge(initiatedTx, left_index=True, right_index=True, how="outer")\
            .merge(plhivipt, left_index=True, right_index=True, how="outer")\
            .merge(childipt, left_index=True, right_index=True, how="outer")\
            .merge(pplcases, left_index=True, right_index=True, how="outer")\
            .merge(ppltx, left_index=True, right_index=True, how="outer")\
            .merge(hivcases, left_index=True, right_index=True, how="outer")\
            .merge(mdrcases, left_index=True, right_index=True, how="outer")\
            .merge(mdrtx, left_index=True, right_index=True, how="outer")\

In [ ]:
outputs.head()

In [ ]:
outputs.to_csv("outputs.csv")

# Activities

Not available: 
* Tests Conducted_value
* Active Case Finding Missions Conducted_value
* Prisoners Screened_value


In [ ]:
# Tests Conducted_value
sicoinRM = pd.read_csv("../../../Resource Tracking/SICOIN GT/IGRC02_PCE_Ejecucion_Fisica.csv")

In [ ]:
sicoinTB = sicoinRM[(sicoinRM.Programa == "17-PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS") ]

In [ ]:
distrMeds = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/Distribución Medicamentos TB/GTM-TB-distribution-2013-2018.csv", index_col=0)

In [ ]:
distrMeds.head()

In [ ]:
meds = distrMeds.groupby(["Year", "code_dept"]).Amount.sum()\
.rename_axis( index = {"Year": "date", "code_dept": "department"})\
.rename("Total Drugs Distributed_value_d")

In [ ]:
distrMeds.Medicine.unique()

In [ ]:
# Isoniazid Distributed_value
isod = distrMeds[distrMeds.Medicine == "ISONIAZIDA"].groupby(["Year", "code_dept"]).Amount.sum()\
.rename_axis( index = {"Year": "date", "code_dept": "department"})\
.rename("Isoniazid Distributed_value_d")

In [ ]:
# PLHIV Screened for TB_value
hivtb = \
sicoinRM[(sicoinRM.Programa == "16-PREVENCIÓN Y CONTROL DE ITS, VIH/SIDA") &
sicoinRM.Resultado.isin(
       ['6.0-Paciente VIH positivo, con prueba diagnóstica de tuberculosis',
       '6.0-Pacientes VIH positivos, con prueba diagnostica de Tuberculosis']
) & (sicoinRM.LugarEjecutado.notna())].assign(date = lambda x: x.Year, municipality = lambda x: x.LugarCod.map(int))\
.assign(department = lambda x: np.floor(x.municipality/100).astype(int) )\
.groupby(["date", "municipality"]).LugarEjecutado.sum()\
.rename("PLHIV Screened for TB_value_m")

In [ ]:
# TB Patients Tested for HIV_value
tbhivTx = sicoinRM[(sicoinRM.Programa == "17-PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS") &
sicoinRM.Resultado.isin(
       ['7.0-Persona con diagnóstico de tuberculosis con prueba diagnóstica de VIH',
       '7.0-Casos de Tuberculosis con prueba diagnóstica de VIH']
) & (sicoinRM.LugarEjecutado.notna())].assign(date = lambda x: x.Year, municipality = lambda x: x.LugarCod.map(int))\
.assign(department = lambda x: np.floor(x.municipality/100).astype(int) )\
.groupby(["date", "municipality"]).LugarEjecutado.sum()\
.rename("TB Patients Tested for HIV_value_m")

In [ ]:
# Number of Cases Screened for MDR_value
mdrSx = sicoinRM[(sicoinRM.Programa == "17-PREVENCIÓN Y CONTROL DE LA TUBERCULOSIS") &
sicoinRM.Resultado.isin(
       ['9.0-Persona con diagnóstico de fracasos al tratamiento de tuberculosis con prueba diagnóstica a TB-MDR (tuberculosis multidrogo resistente)',
        '9.0-Casos de fracasos a tratamiento de Tuberculosis con prueba diagnóstica a TB-MDR']
) & (sicoinRM.LugarEjecutado.notna())].assign(date = lambda x: x.Year, municipality = lambda x: x.LugarCod.map(int))\
.assign(department = lambda x: np.floor(x.municipality/100).astype(int) )\
.groupby(["date", "municipality"]).LugarEjecutado.sum()\
.rename("Number of Cases Screened for MDR_value_m")

In [ ]:
cohortsExtraD = pd.read_csv("../../../Outcome Measurement Data/TUBERCULOSIS/COHORTES/GTM - Tx Cohort Data - Extra tables.csv")

In [ ]:
cohortsExtraD.columns

In [ ]:
cohortsExtraD.table.unique()

In [ ]:
cohortes.table.unique()

In [ ]:
cohortes[cohortes.table.isin(['MDR sensitivity tests on new','MDR test on previously treated']) & 
        (cohortes.deptocode==0)]

In [ ]:
mdrSx = cohortes[cohortes.table.isin(['MDR sensitivity tests on new','MDR test on previously treated']) & 
        (cohortes.deptocode!=0)].groupby(["year", "deptocode"]).value.sum()\
            .rename_axis(index={ "year": "date", "deptocode": "department" })\
            .rename("Number of Cases Screened for MDR_value_d")

In [ ]:
# Second Line Drugs Distributed_value

In [ ]:
distrMeds.Medicine.unique()

In [ ]:
# De acuerdo al manual de tb drogoresistente del programa
segundaLinea = [
    'LEVOFLOXACINA',
    'CAPREOMICINA',
    'MOXIFLOXACINA',
    'KANAMICINA',
    'ESTREPTOMICINA',
    'ETHIONAMIDA',
    'CICLOCERINA',
    'CICLOSERINA',
    'LINEZOLID',
    'CLOFAZIMINE',
    'PIRAZINAMIDA',
    'ETAMBUTOL',
    'ACIDO PARAAMINOSALICILICO',
    'IMIPENEM/CILASTATINA',
    'AMOXICILINA/ACIDO CLAVULANICO',
]

In [ ]:
meds2dl = distrMeds[distrMeds.Medicine.isin(segundaLinea)].groupby(["Year", "code_dept"]).Amount.sum()\
.rename_axis( index = {"Year": "date", "code_dept": "department"})\
.rename("Total Drugs Distributed_value_d")

In [ ]:
activities = pd.merge(meds, isod, left_index=True, right_index=True, how="outer")\
            .merge(hivtb, left_index=True, right_index=True, how="outer")\
            .merge(tbhivTx, left_index=True, right_index=True, how="outer")\
            .merge(mdrSx, left_index=True, right_index=True, how="outer")\
            .merge(meds2dl, left_index=True, right_index=True, how="outer")\
            

In [ ]:
activities

In [ ]:
activities.to_csv("activities.csv")

# Impact

In [ ]:
# Worldpop estimations are in the helper functions 
import os

os.chdir("../../core/")
import GT_helper_functions as hf
import GT_load_data as data

In [ ]:
plt.plot(list(range(2010,2030) ), [ hf.GTMuniPopulation(101,y) for y in range(2010,2030)] )
None

In [ ]:
notifsRate = casesNotif.reset_index().apply(lambda x: x["Cases Notified_value_m"] / hf.GTMuniPopulation(x.municipality, x.date), 1).iloc[:,0]

In [ ]:
notifsRate.index = casesNotif.index.copy()

In [ ]:
notifsRate = notifsRate.rename("Case Notification Rate_value")

In [ ]:
# Mortality Rate_value
vrredis =  pd.read_csv("../../Outcome Measurement Data/MULTI/VR/vr_redistributed_collapsed_diseases.csv", index_col = 0)

In [ ]:
munis_ids = pd.read_csv("../../Outcome Measurement Data/MULTI/VR/GTM_muni_merges_2009_2016_fixed.csv", index_col = 0)

In [ ]:
vrredis = vrredis.merge(munis_ids[["adm2_country_code", "adm2_gbd_id"]], left_on= "location_id", right_on="adm2_gbd_id")

In [114]:
mortcounts = vrredis[vrredis.disease.isin(["TB", "TB-VIH"]) & (vrredis.adm2_country_code.notna())]\
.assign(municipality = lambda x: x.adm2_country_code.astype(int), date = lambda x: x.year_id,
        department = lambda x: np.floor(x.adm2_country_code/100).astype(int) ) \
.groupby(["date", "department", "municipality"]).deaths.sum().rename("mortcounts")

In [116]:
mortrate = mortcounts.reset_index().apply(lambda x: x["mortcounts"] / hf.GTMuniPopulation(x.municipality, x.date), 1).iloc[:,0]

In [117]:
mortrate.index = mortcounts.index

In [103]:
#  Treatment Success Rate_value
txsrate = pivot_cohortes["COMPLETED TREATMENT"].divide(pivot_cohortes.sum(1))\
.rename_axis(index={"deptocode": "department", "year": "date"}).rename("Treatment Success Rate_value")

In [108]:
# HIV/TB Treatment Success Rate_value
pivot_cohortes_tbhiv = cohortes[(cohortes.col_name.isin(["TOTAL"])) & 
       (cohortes.table.isin(["Nuevos TB/VIH"])) &
       (cohortes.deptocode!=0)].\
    groupby(["year", "deptocode", "row_name_B"]).value.sum().unstack(2)
tbhiv_txsrate = pivot_cohortes_tbhiv["COMPLETED TREATMENT"].divide(pivot_cohortes_tbhiv.sum(1))\
.rename_axis(index={"deptocode": "department", "year": "date"}).rename("Treatment Success Rate_value")

In [119]:
# HIV/TB Mortality Rate_value
tbhiv_mortrate = vrredis[vrredis.disease.isin(["TB", "TB-VIH"]) & (vrredis.adm2_country_code.notna())]\
.assign(municipality = lambda x: x.adm2_country_code.astype(int), date = lambda x: x.year_id,
        department = lambda x: np.floor(x.adm2_country_code/100).astype(int) ) \
.groupby(["date", "department", "municipality"]).deaths.sum().rename("HIV/TB Mortality Rate_value")

In [122]:
# MDR Treatment Success Rate_value
mdrtb.columns

Index(['Unnamed: 0', 'ID', 'X_0', 'Gender', 'Age', 'X_1', 'Muni', 'Depto',
       'DAS', 'PxNew', 'PxRelapse', 'PxLost', 'TxFailure', 'TBType', 'PDSDate',
       'PDSResult', 'X_2', 'X_3', 'TxDate', 'DateBk_1', 'ResultBk_1',
       'DateCulture_1', 'ResultCulture_1', 'DateCulture_2', 'ResultCulture_2',
       'DateCulture_3', 'ResultCulture_3', 'DateCulture_4', 'ResultCulture_4',
       'DateCulture_5', 'ResultCulture_5', 'DateCulture_6', 'ResultCulture_6',
       'DateCulture_7', 'ResultCulture_7', 'DateCulture_8', 'ResultCulture_8',
       'DateCulture_9', 'ResultCulture_9', 'DateCulture_10',
       'ResultCulture_10', 'DateCulture_11', 'ResultCulture_11',
       'DateCulture_12', 'ResultCulture_12', 'DateCulture_13',
       'ResultCulture_13', 'DateCulture_14', 'ResultCulture_14',
       'DateCulture_15', 'ResultCulture_15', 'DateCulture_16',
       'ResultCulture_16', 'DateCulture_17', 'ResultCulture_17',
       'DateCulture_18', 'ResultCulture_18', 'DateCulture_19',
       'Result

In [125]:
mdrtb.ExitCondition.value_counts(dropna=False)

NaN                           86
ambulatorio                   23
FALLECIDO                      7
PERDIDA EN EL SEGUIMIENTO      5
AMBULATORIO                    5
fallecido                      4
FRACASO                        4
Fallecido                      4
FALLECIDA                      3
TX. COMPLETO                   2
PÉRDIDA EN EL SEGUIMIENTO      2
vivo                           2
FUGADO                         2
 EGRESO CONTRAINDICADO         1
ABANDONÓ                       1
CONTRAINDICADO                 1
viva                           1
FALLECIDA                      1
Fallecida                      1
Vivo                           1
Fallecida                      1
Name: ExitCondition, dtype: int64

In [133]:
mdrtb[["ResultCulture_1", "ResultCulture_2", "ResultCulture_3", "ResultCulture_4", "ResultCulture_5"]].fillna("NAN")\
.groupby(["ResultCulture_3", "ResultCulture_4", "ResultCulture_5"]).apply(len)

ResultCulture_3    ResultCulture_4      ResultCulture_5  
.+                 negativo             negativo              1
.++                NAN                  NAN                   1
CONTAMINADO        CONTAMINADO          NAN                   1
                                        NEGATIVO              2
                   POSITIVO             POSITIVO              1
CONTAMINADO        Negativo             NEGATIVO              1
NAN                CONTAMINADO          NAN                   1
                   NAN                  CONTAMINADO           1
                                        NAN                  69
                                        NEGATIVO              7
                                        POSITIVO              1
                                        negativo              2
                   NEGATIVO             NEGATIVO              2
                   negativo             NAN                   2
                                        NEGATI